# дефолт

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import shutil
# shutil.copy('/content/drive/MyDrive/new_hack_1/data_members.csv', '/content/data_members.csv')
shutil.copy('/content/drive/MyDrive/new_hack_1/data_resume.csv', '/content/data_resume.csv')
shutil.copy('/content/drive/MyDrive/new_hack_1/df_emb.csv', '/content/df_emb.csv')

'/content/df_emb.csv'

# препроцесинг

In [ ]:
with open('case_2_data_for_members.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
data_members = pd.DataFrame(list(data))
data_members.head(5)

In [59]:
data_members_vacancy = pd.DataFrame(list(data_members['vacancy']))
data_members_vacancy.head(5)

,uuid,name,keywords,description,comment
0,779f3a59-206a-3241-adc4-d7db504f960b,Java разработчик команда Инвестиции,None,Описание Мы расширяем команды и ищем разработ...,450 на руки
1,7a4813fc-43bc-3896-a607-4c8682b01002,Системный аналитик,None,Уровень: СА уровня Middle+/Senior от 3х лет В...,350 000 - 400 000 гросс
2,c03085c3-9b1e-3564-bb1e-59aa72e5fbca,Ведущий/ Главный аналитик DWH,None,Желательно знания Oracle сегодня — не прост...,"удаленка, 300к гросс (может +10%) + 2 оклада п..."
3,a8dd83c3-178d-3c70-90c2-7c3648f6b96a,Системный аналитик,None,Знания и опыт • Умение управлять ожиданиями з...,None
4,9d98eba0-13bb-38d3-b742-4fd445954b3d,Product manager,None,"- Продактов в компании сейчас порядка 250, вс...",None


In [201]:
df = pd.DataFrame(data_members['failed_resumes'][0])
df['vacancy'] = data_members['vacancy'][0]['uuid']
for i in range(1, 29):
  df2 = pd.DataFrame(data_members['failed_resumes'][i])
  df2['vacancy'] = data_members['vacancy'][i]['uuid']
  df = pd.concat([df, df2])
df['prinali'] = np.zeros(len(df))

In [202]:
df_confirmed_resumes = pd.DataFrame(data_members['confirmed_resumes'][0])
df_confirmed_resumes['vacancy'] = data_members['vacancy'][0]['uuid']
for i in range(1, 29):
  df2 = pd.DataFrame(data_members['confirmed_resumes'][i])
  df2['vacancy'] = data_members['vacancy'][i]['uuid']
  df_confirmed_resumes = pd.concat([df_confirmed_resumes, df2])
df_confirmed_resumes['prinali'] = np.ones(len(df_confirmed_resumes))

In [203]:
df = pd.concat([df_confirmed_resumes, df])
data_members_vacancy.columns = ['vacancy', 'name', 'keywords', 'description', 'comment']
df = pd.merge(df, data_members_vacancy, on='vacancy')

In [204]:
df = df.drop(columns = ['uuid', 'first_name', 'last_name', 'vacancy', 'keywords'])

In [205]:
df.head(3)

,birth_date,country,city,about,key_skills,experienceItem,educationItem,languageItem,prinali,languageItems,name,description,comment
0,1987-01-01,Россия,Великий Новгород,None,"Java, Java Spring Framework, Hibernate, Git, P...","[{'starts': '2022-02-01', 'ends': None, 'emplo...","[{'year': None, 'organization': 'НовГУ', 'facu...",NaN,1.0,NaN,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки
1,1988-12-29,Россия,Tolyatti,None,"MATLAB, Java, Android SDK, Design Patterns, Ec...","[{'starts': '2021-01-11', 'ends': None, 'emplo...","[{'year': 2011, 'organization': 'Samara State ...","[Английский, Русский]",1.0,NaN,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки
2,1990-01-01,Россия,Каспийск,"Усидчивый и терпеливый программист, обладающи...","Java – 17, Spring Boot 2.7, Liquibase, Spring ...","[{'starts': '2022-06-01', 'ends': None, 'emplo...","[{'year': 2013, 'organization': 'ДГТУ-Махачкал...",NaN,1.0,NaN,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки


In [206]:
df['experienceItem_'] = np.zeros(len(df))
for i in range(len(df)):
  d = []
  try:
    for j in range(len(df['experienceItem'][i])):
      start = pd.to_datetime(df['experienceItem'][i][j]['starts'])
      end =  pd.to_datetime(df['experienceItem'][i][j]['ends'])
      if start == None or end == None:
        duration = None
      else:
        duration = end - start
        d.append(duration.days / 364)

  except:
    print(i, "зафейлилось")
  if d:
    df['experienceItem_'][i] = int(np.sum(d))
    # print(d)
  else:
    df['experienceItem_'][i] = None

<ipython-input-206-9b2ac7cea9a3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['experienceItem_'][i] = int(np.sum(d))
<ipython-input-206-9b2ac7cea9a3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['experienceItem_'][i] = None


160 зафейлилось


In [207]:
df = df.fillna(0)

In [208]:
def f(x):
  s = {'comment':['350 000 - 400 000 гросс', '450 на руки', 'удаленка, 300к гросс (может +10%) + 2 оклада премия годовая в среднем', 'смотрим до 500 net окладом + perf review каждые 6 мес. (1,5-2 оклада после ревью премию/ 1,5 для senior, 2 для lead)'],
       'comment_int': [375, 450, 300, 500]}
  try:
    return s['comment_int'][s['comment'].index(x)]
  except:
    return 0

In [209]:
df['comment'] = df['comment'].apply(f)

In [210]:
df['birth_date'] = pd.to_datetime(df['birth_date'], format='%Y-%m-%d')
target_date = pd.datetime(2024, 2, 28)
df['age'] = (target_date - df['birth_date']).astype('<m8[Y]')

<ipython-input-210-7072b0ad37d8>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  target_date = pd.datetime(2024, 2, 28)


In [212]:
df = df.drop(columns='birth_date')

In [213]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['country'] = label_encoder.fit_transform(df['country'])
# df['city'] = label_encoder.fit_transform(df['city'])

In [121]:
df['Русский яз'] = np.zeros(len(df))
df['Английский яз'] = np.zeros(len(df))
for i in range(len(df)):
  if df['languageItem'][i] != 0 and df['languageItem'][i] != None:
    if "Английский" in df['languageItem'][i]:
      df['Английский яз'][i] = 1
    if "Русский" in df['languageItem'][i]:
      df['Русский яз'][i] = 1

<ipython-input-121-876a1cd25875>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Английский яз'][i] = 1
<ipython-input-121-876a1cd25875>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Русский яз'][i] = 1


In [123]:
df.head(3)

,country,city,about,key_skills,experienceItem,educationItem,languageItem,prinali,languageItems,name,description,comment,experienceItem_,age,experienceItem_text,Русский яз,Английский яз
0,5,13,0,"Java, Java Spring Framework, Hibernate, Git, P...","[{'starts': '2022-02-01', 'ends': None, 'emplo...","[{'year': None, 'organization': 'НовГУ', 'facu...",0,1.0,0,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450,4.0,37.0,Проекты: Внутренний продукт а по заказу инст...,0.0,0.0
1,5,4,0,"MATLAB, Java, Android SDK, Design Patterns, Ec...","[{'starts': '2021-01-11', 'ends': None, 'emplo...","[{'year': 2011, 'organization': 'Samara State ...","[Английский, Русский]",1.0,0,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450,9.0,35.0,Development and testing (unit and integration...,1.0,1.0
2,5,35,"Усидчивый и терпеливый программист, обладающи...","Java – 17, Spring Boot 2.7, Liquibase, Spring ...","[{'starts': '2022-06-01', 'ends': None, 'emplo...","[{'year': 2013, 'organization': 'ДГТУ-Махачкал...",0,1.0,0,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450,8.0,34.0,Проект: ДБО в команду ведение счета. Задачи: ...,0.0,0.0


# ембендинги

In [228]:
df.drop(columns=["country", "city", "about", "key_skills", "experienceItem", "educationItem", "name", "description", "languageItem"]).head(3)

,prinali,languageItems,comment,experienceItem_,age,experienceItem_text
0,1.0,0,450,4.0,37.0,Проекты: Внутренний продукт а по заказу инст...
1,1.0,0,450,9.0,35.0,Development and testing (unit and integration...
2,1.0,0,450,8.0,34.0,Проект: ДБО в команду ведение счета. Задачи: ...


In [211]:
df['experienceItem_text'] = np.zeros(len(df))
for i in range(len(df)):
  d = []
  try:
    for j in range(len(df['experienceItem'][i])):
      if df['experienceItem'][i][j]['description'] != None:
        d.append(df['experienceItem'][i][j]['description'])
  except:
    print(i, "анлак")
  if d:
    df['experienceItem_text'][i] = " ".join(d)

160 анлак


<ipython-input-211-8fa9aaeaccd8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['experienceItem_text'][i] = " ".join(d)


In [124]:
for i in range(len(df)):
  if df['about'][i] != 0 and df['about'][i] != None:
    df['experienceItem_text'][i] += df['about'][i]

<ipython-input-124-f06709edc61f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['experienceItem_text'][i] += df['about'][i]


In [125]:
import re
import string
def clean_text(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    return text
df['experienceItem_text'] = df['experienceItem_text'].apply(clean_text)

In [126]:
import nltk

stemmer = nltk.SnowballStemmer("russian")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text
df['experienceItem_text'] = df['experienceItem_text'].apply(stemm_text)

In [229]:
drops = ["country", "city", "about", "key_skills", "experienceItem", "educationItem", "name", "description", "languageItem", "languageItems", 'experienceItem_text']
df.rename(columns={'prinali': 'target'}, inplace=True)

In [150]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['experienceItem_text'])
Y = df['target']
# Y = pd.get_dummies(df['target'], columns=['target'])

In [140]:
emb = pd.DataFrame(X.toarray())
df_with_embeddings = pd.concat([df.drop(columns=drops), emb], axis=1)
df_with_embeddings = df_with_embeddings.applymap(lambda x: 0 if pd.isna(x) else x)

In [258]:
df_with_embeddings = df_with_embeddings.drop(columns = ["country", "city"])

In [259]:
X_ = df_with_embeddings.drop(columns='target')
Y_ = df_with_embeddings['target']

In [260]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_.values, Y_.values, test_size = 0.2, random_state = 42)

In [151]:
# X_train, X_val, y_train, y_val = train_test_split(X.toarray(), Y.values, test_size = 0.2, random_state = 42)

In [261]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state = 42)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [262]:
from sklearn.metrics import classification_report, accuracy_score
# print(classification_report(model.predict(X_val.values), y_val))
print(accuracy_score(y_val, model.predict(X_val)))

0.8636363636363636


In [265]:
from xgboost import XGBClassifier
model = XGBClassifier(booster='gbtree', max_depth=15, n_jobs=4, n_estimators=500)
model.fit(X_train, y_train)
# print(classification_report(model.predict(X_val), y_val))
print(accuracy_score(y_val, model.predict(X_val)))

0.8560606060606061


# тестовый датасет

In [300]:
import json
from pandas import json_normalize
with open('case_2_reference_without_resume_sorted.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
data_resume = pd.DataFrame(data['resumes'])
data_resume.head(3)

,uuid,first_name,last_name,birth_date,country,city,about,key_skills,experienceItem,languageItems,educationItem,languageItem
0,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33,Клим,Тетерина,1991-05-23,Россия,Москва,"Личные качества: ответственность, лидерство, г...","Java Core, Spring Framework, Hibernate ORM, Po...","[{'starts': '2022-10-01', 'ends': None, 'emplo...","[Русский, Английский]","[{'year': 2021, 'organization': 'Московский эн...",NaN
1,f8b69e24-e2c0-3186-9578-380835eb2ee7,Алиса,Ситникова,1990-07-18,Россия,Нижний Новгород,Обязанности: в составе группы поддержки я отв...,"Java 8-17, Java SE, Java EE, Spring Framework ...","[{'starts': '2019-01-29', 'ends': None, 'emplo...",NaN,NaN,NaN
2,e3976e74-e71b-34db-8e98-08dc422fa567,Розалина,Андреев,1990-01-01,Россия,Санкт-Петербург,Обучаемость Коммуникабельность Организованнос...,"Java 8 и 11, osgi, postgresql, testng, mockito...","[{'starts': '2019-09-04', 'ends': '2022-02-04'...","[Русский, Английский]","[{'year': 2009, 'organization': 'Архангельский...",NaN


In [301]:
data_vacancy = pd.DataFrame([data['vacancy']])
data_vacancy.head(3)

,uuid,name,keywords,description,comment
0,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,Java разработчик,"Kafka, Java, RxJava, Hystrix, MongoDB",Требования: 4+ года опыта работы с Java 8+ или...,None


In [304]:
data_resume['lol'] = np.zeros(len(data_resume))
data_vacancy['lol'] = np.zeros(len(data_vacancy))

In [306]:
data_vacancy.columns = ['vacancy', 'name', 'keywords', 'description', 'comment', "lol"]
data_resume = pd.merge(data_resume, data_vacancy, on='lol')
data_resume = data_resume.drop(columns = ['first_name', 'vacancy', 'last_name', 'keywords', 'lol'])

In [307]:
data_resume.head(3)

,uuid,birth_date,country,city,about,key_skills,experienceItem,languageItems,educationItem,languageItem,name,description,comment
0,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33,1991-05-23,Россия,Москва,"Личные качества: ответственность, лидерство, г...","Java Core, Spring Framework, Hibernate ORM, Po...","[{'starts': '2022-10-01', 'ends': None, 'emplo...","[Русский, Английский]","[{'year': 2021, 'organization': 'Московский эн...",NaN,Java разработчик,Требования: 4+ года опыта работы с Java 8+ или...,None
1,f8b69e24-e2c0-3186-9578-380835eb2ee7,1990-07-18,Россия,Нижний Новгород,Обязанности: в составе группы поддержки я отв...,"Java 8-17, Java SE, Java EE, Spring Framework ...","[{'starts': '2019-01-29', 'ends': None, 'emplo...",NaN,NaN,NaN,Java разработчик,Требования: 4+ года опыта работы с Java 8+ или...,None
2,e3976e74-e71b-34db-8e98-08dc422fa567,1990-01-01,Россия,Санкт-Петербург,Обучаемость Коммуникабельность Организованнос...,"Java 8 и 11, osgi, postgresql, testng, mockito...","[{'starts': '2019-09-04', 'ends': '2022-02-04'...","[Русский, Английский]","[{'year': 2009, 'organization': 'Архангельский...",NaN,Java разработчик,Требования: 4+ года опыта работы с Java 8+ или...,None


In [308]:
data_resume['experienceItem_'] = np.zeros(len(data_resume))
for i in range(len(data_resume)):
  d = []
  try:
    for j in range(len(data_resume['experienceItem'][i])):
      start = pd.to_datetime(data_resume['experienceItem'][i][j]['starts'])
      end =  pd.to_datetime(data_resume['experienceItem'][i][j]['ends'])
      if start == None or end == None:
        duration = None
      else:
        duration = end - start
        d.append(duration.days / 364)

  except:
    print(i, "зафейлилось")
  if d:
    data_resume['experienceItem_'][i] = int(np.sum(d))
    # print(d)
  else:
    data_resume['experienceItem_'][i] = None

<ipython-input-308-a47105beffb3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_resume['experienceItem_'][i] = int(np.sum(d))
<ipython-input-308-a47105beffb3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_resume['experienceItem_'][i] = None


In [309]:
data_resume = data_resume.fillna(0)
def f(x):
  s = {'comment':['350 000 - 400 000 гросс', '450 на руки', 'удаленка, 300к гросс (может +10%) + 2 оклада премия годовая в среднем', 'смотрим до 500 net окладом + perf review каждые 6 мес. (1,5-2 оклада после ревью премию/ 1,5 для senior, 2 для lead)'],
       'comment_int': [375, 450, 300, 500]}
  try:
    return s['comment_int'][s['comment'].index(x)]
  except:
    return 0
data_resume['comment'] = data_resume['comment'].apply(f)

In [310]:
data_resume['birth_date'] = pd.to_datetime(data_resume['birth_date'], format='%Y-%m-%d')
target_date = pd.datetime(2024, 2, 28)
data_resume['age'] = (target_date - data_resume['birth_date']).astype('<m8[Y]')

<ipython-input-310-38bbdcafb435>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  target_date = pd.datetime(2024, 2, 28)


In [311]:
data_resume['experienceItem_text'] = np.zeros(len(data_resume))
for i in range(len(data_resume)):
  d = []
  try:
    for j in range(len(data_resume['experienceItem'][i])):
      if data_resume['experienceItem'][i][j]['description'] != None:
        d.append(data_resume['experienceItem'][i][j]['description'])
  except:
    print(i, "анлак")
  if d:
    data_resume['experienceItem_text'][i] = " ".join(d)

<ipython-input-311-dc972f09a13d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_resume['experienceItem_text'][i] = " ".join(d)


In [312]:
data_resume = data_resume.drop(columns='country')
data_resume = data_resume.drop(columns='city')

In [313]:
data_resume['Русский яз'] = np.zeros(len(data_resume))
data_resume['Английский яз'] = np.zeros(len(data_resume))
for i in range(len(data_resume)):
  if data_resume['languageItem'][i] != 0 and data_resume['languageItem'][i] != None:
    if "Английский" in data_resume['languageItem'][i]:
      data_resume['Английский яз'][i] = 1
    if "Русский" in data_resume['languageItem'][i]:
      data_resume['Русский яз'][i] = 1

<ipython-input-313-b6b3fb67ff3f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_resume['Английский яз'][i] = 1
<ipython-input-313-b6b3fb67ff3f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_resume['Русский яз'][i] = 1


In [318]:
data_resume.head(3)

,uuid,birth_date,about,key_skills,experienceItem,languageItems,educationItem,languageItem,name,description,comment,experienceItem_,age,experienceItem_text,Русский яз,Английский яз
0,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33,1991-05-23,"Личные качества: ответственность, лидерство, г...","Java Core, Spring Framework, Hibernate ORM, Po...","[{'starts': '2022-10-01', 'ends': None, 'emplo...","[Русский, Английский]","[{'year': 2021, 'organization': 'Московский эн...",0,Java разработчик,Требования: 4+ года опыта работы с Java 8+ или...,0,3.0,32.0,руковод команд разработчик в том числ 6 бэкенд...,0.0,0.0
1,f8b69e24-e2c0-3186-9578-380835eb2ee7,1990-07-18,Обязанности: в составе группы поддержки я отв...,"Java 8-17, Java SE, Java EE, Spring Framework ...","[{'starts': '2019-01-29', 'ends': None, 'emplo...",0,0,0,Java разработчик,Требования: 4+ года опыта работы с Java 8+ или...,0,6.0,33.0,проект по поддержк инфраструктур информацион ...,0.0,0.0
2,e3976e74-e71b-34db-8e98-08dc422fa567,1990-01-01,Обучаемость Коммуникабельность Организованнос...,"Java 8 и 11, osgi, postgresql, testng, mockito...","[{'starts': '2019-09-04', 'ends': '2022-02-04'...","[Русский, Английский]","[{'year': 2009, 'organization': 'Архангельский...",0,Java разработчик,Требования: 4+ года опыта работы с Java 8+ или...,0,5.0,34.0,анализ целев сист sap mdm hcm hana abap ume d...,0.0,0.0


In [315]:
for i in range(len(data_resume)):
  if data_resume['about'][i] != 0 and data_resume['about'][i] != None:
    data_resume['experienceItem_text'][i] += data_resume['about'][i]

<ipython-input-315-a3f80a3518a6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_resume['experienceItem_text'][i] += data_resume['about'][i]


In [316]:
import re
import string
def clean_text(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    return text
data_resume['experienceItem_text'] = data_resume['experienceItem_text'].apply(clean_text)

In [317]:
import nltk

stemmer = nltk.SnowballStemmer("russian")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text
data_resume['experienceItem_text'] = data_resume['experienceItem_text'].apply(stemm_text)

In [245]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorizer = TfidfVectorizer()
X = vectorizer.transform(data_resume['experienceItem_text'])

In [319]:
drops = ['about',
         'key_skills',
         'experienceItem',
         'educationItem',
         'name',
         'description',
         'languageItem',
         'languageItems',
         'experienceItem_text']

In [320]:
ans_df = data_resume.drop(columns=drops)

In [249]:
x_emb = pd.DataFrame(X.toarray())
X_embeddings = pd.concat([data_resume.drop(columns=drops), x_emb], axis=1)
X_embeddings = X_embeddings.applymap(lambda x: 0 if pd.isna(x) else x)

In [266]:
X_embeddings.head(3)

,comment,experienceItem_,age,Русский яз,Английский яз,0,1,2,3,4,...,14557,14558,14559,14560,14561,14562,14563,14564,14565,14566
0,0,3.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,6.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,5.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# предикт

In [269]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state = 42)
model.fit(X_.values, Y_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [324]:
ans_df['ans'] = model.predict_proba(X_embeddings.values)[:, 1]

In [ ]:
ans_df = ans_df[['uuid', 'ans']]

In [336]:
sorted_ans_df = ans_df.sort_values('ans', ascending=False)

In [338]:
sorted_ans_df.head(5)

,uuid,ans
59,fd0ccbd0-3a58-3818-8691-98f31de17527,0.993985
26,a71b0749-1ebc-3099-b0a1-342ca64d1575,0.991301
39,fd9c4130-177f-3546-8974-189a52fcc496,0.990988
36,b1357f8e-3c98-3142-81e7-fd729ec50ca0,0.989806
68,6f48fd66-a056-3172-af60-632f22844934,0.989084


In [340]:
sorted_ans_df.to_csv('sorted_ans_df.csv', index=False)
shutil.move('/content/sorted_ans_df.csv', '/content/drive/MyDrive/new_hack_1/sorted_ans_df.csv')

'/content/drive/MyDrive/new_hack_1/sorted_ans_df.csv'

# не запускать там word2vec

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import word2vec

texsts = [tweet.split() for tweet in df['experienceItem_text'].values]
w2v = word2vec.Word2Vec(texsts, workers=4, vector_size=100, min_count=10, window=3)
def sentence_emb(text, model=w2v.wv):
    n = 0
    em = np.zeros(100)
    for word in text.split():
        if word in model:
            em += np.array(model[word])
            n+=1
    if n:
        em = em/n
    return em
df_ = df['experienceItem_text'].apply(sentence_emb)
df_ = pd.DataFrame(list(df_), columns = [i for i in range(100)])

df_['target'] = df['target']

df_ = df_.drop(columns='target')
df_ = pd.concat([df_with_embeddings, emb], axis=1)
df_ = df_.applymap(lambda x: 0 if pd.isna(x) else x)